In [1]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec, clean_str
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine

from tqdm import tqdm

In [2]:
dataset = 'NTIS'

word_embeddings_dim = 300
word_vector_map = {}

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []

print("파일 여는 중")
f = open('data/' + dataset + '.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_name_list.append(line.strip())
    temp = line.split("\t")
    if temp[1].find('test') != -1:
        doc_test_list.append(line.strip())
    elif temp[1].find('train') != -1:
        doc_train_list.append(line.strip())
f.close()
print("종료")
# print(doc_train_list)
# print(doc_test_list)

파일 여는 중
종료


In [3]:
doc_test_list
doc_train_list

['0\ttrain\tLC05',
 '1\ttrain\tLC05',
 '2\ttrain\tLC01',
 '3\ttrain\tLC01',
 '4\ttrain\tLC05',
 '5\ttrain\tLC05',
 '6\ttrain\tLC06',
 '7\ttrain\tND04',
 '8\ttrain\tEB02',
 '9\ttrain\tEE13',
 '10\ttrain\tND07',
 '11\ttrain\tND07',
 '12\ttrain\tND07',
 '13\ttrain\tEE07',
 '14\ttrain\tEI01',
 '15\ttrain\tND07',
 '16\ttrain\tND07',
 '17\ttrain\tEI11',
 '18\ttrain\tEI11',
 '19\ttrain\tEI11',
 '20\ttrain\tEH10',
 '21\ttrain\tEH10',
 '22\ttrain\tND07',
 '23\ttrain\tEI11',
 '24\ttrain\tEI01',
 '25\ttrain\tEI01',
 '26\ttrain\tEI03',
 '27\ttrain\tEI03',
 '28\ttrain\tEI03',
 '29\ttrain\tND07',
 '30\ttrain\tND07',
 '31\ttrain\tEI11',
 '32\ttrain\tEI02',
 '33\ttrain\tEI02',
 '34\ttrain\tEI09',
 '35\ttrain\tEI09',
 '36\ttrain\tND07',
 '37\ttrain\tND07',
 '38\ttrain\tED08',
 '39\ttrain\tED08',
 '40\ttrain\tEA14',
 '41\ttrain\tEE13',
 '42\ttrain\tEA14',
 '43\ttrain\tEA05',
 '44\ttrain\tEA14',
 '45\ttrain\tEA14',
 '46\ttrain\tEH03',
 '47\ttrain\tEH03',
 '48\ttrain\tLB02',
 '49\ttrain\tLB02',
 '50\ttrai

In [4]:
doc_content_list = []
f = open('data/corpus/' + dataset + '.txt', 'r')
lines = f.readlines()
for line in tqdm(lines):
    doc_content_list.append(line.strip())
f.close()
# print(doc_content_list)

train_ids = []
for train_name in tqdm(doc_train_list):
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
print(train_ids[:1000])
random.shuffle(train_ids)

# partial labeled data
#train_ids = train_ids[:int(0.2 * len(train_ids))]

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('data/' + dataset + '.train.index', 'w')
f.write(train_ids_str)
f.close()

100%|██████████| 188662/188662 [04:58<00:00, 631.83it/s]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [5]:
test_ids = []
for test_name in tqdm(doc_test_list):
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
print(test_ids[:1000])
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('data/' + dataset + '.test.index', 'w')
f.write(test_ids_str)
f.close()

100%|██████████| 47165/47165 [03:23<00:00, 232.13it/s]


[188662, 188663, 188664, 188665, 188666, 188667, 188668, 188669, 188670, 188671, 188672, 188673, 188674, 188675, 188676, 188677, 188678, 188679, 188680, 188681, 188682, 188683, 188684, 188685, 188686, 188687, 188688, 188689, 188690, 188691, 188692, 188693, 188694, 188695, 188696, 188697, 188698, 188699, 188700, 188701, 188702, 188703, 188704, 188705, 188706, 188707, 188708, 188709, 188710, 188711, 188712, 188713, 188714, 188715, 188716, 188717, 188718, 188719, 188720, 188721, 188722, 188723, 188724, 188725, 188726, 188727, 188728, 188729, 188730, 188731, 188732, 188733, 188734, 188735, 188736, 188737, 188738, 188739, 188740, 188741, 188742, 188743, 188744, 188745, 188746, 188747, 188748, 188749, 188750, 188751, 188752, 188753, 188754, 188755, 188756, 188757, 188758, 188759, 188760, 188761, 188762, 188763, 188764, 188765, 188766, 188767, 188768, 188769, 188770, 188771, 188772, 188773, 188774, 188775, 188776, 188777, 188778, 188779, 188780, 188781, 188782, 188783, 188784, 188785, 188786,

In [6]:
ids = train_ids + test_ids
print(ids[:1000])
print(len(ids))

[114393, 95627, 181101, 161263, 174467, 15725, 160802, 123775, 53921, 92393, 76477, 152816, 55038, 154208, 21133, 124135, 172786, 184951, 117451, 173477, 35522, 95967, 185530, 181267, 26669, 50792, 115257, 86507, 123575, 58942, 80922, 7275, 123862, 157838, 91683, 105428, 152200, 77376, 109328, 140818, 75522, 14668, 1374, 23121, 134653, 148978, 110746, 181180, 22085, 175665, 93935, 146609, 156399, 153843, 119544, 30743, 181851, 73832, 87515, 54283, 41753, 156166, 159598, 60439, 20080, 57490, 136039, 60352, 137582, 78530, 158091, 61594, 111143, 103208, 161648, 117162, 186109, 150500, 158961, 155607, 182764, 64605, 98918, 164959, 179754, 159057, 30180, 171230, 106451, 127873, 8750, 12547, 36040, 70553, 41935, 15639, 18537, 9625, 127747, 17059, 28399, 52713, 38377, 58442, 2422, 129332, 45559, 151465, 63155, 179155, 162286, 149114, 178261, 89222, 106612, 67063, 123235, 106226, 4234, 138239, 114109, 21241, 182922, 168677, 89042, 175402, 113852, 120768, 141392, 161477, 176514, 108513, 174243,

In [7]:
len(doc_content_list)

235827

In [8]:
shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in tqdm(ids):
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('data/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_name_str)
f.close()

f = open('data/corpus/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()

100%|██████████| 235827/235827 [00:00<00:00, 632963.07it/s]


In [10]:
# build vocab
word_freq = {}
word_set = set()
for doc_words in tqdm(shuffle_doc_words_list):
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

100%|██████████| 235827/235827 [00:22<00:00, 10369.82it/s]


In [11]:
word_doc_list = {}

for i in tqdm(range(len(shuffle_doc_words_list))):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)

100%|██████████| 235827/235827 [00:31<00:00, 7603.76it/s]


In [12]:
word_doc_freq = {}
for word, doc_list in tqdm(word_doc_list.items()):
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for i in tqdm(range(vocab_size)):
    word_id_map[vocab[i]] = i

vocab_str = '\n'.join(vocab)

f = open('data/corpus/' + dataset + '_vocab.txt', 'w')
f.write(vocab_str)
f.close()

100%|██████████| 1793851/1793851 [00:01<00:00, 1243366.72it/s]


In [ ]:
'''
Word definitions begin
'''
'''
definitions = []

for word in vocab:
    word = word.strip()
    synsets = wn.synsets(clean_str(word))
    word_defs = []
    for synset in synsets:
        syn_def = synset.definition()
        word_defs.append(syn_def)
    word_des = ' '.join(word_defs)
    if word_des == '':
        word_des = '<PAD>'
    definitions.append(word_des)

string = '\n'.join(definitions)


f = open('data/corpus/' + dataset + '_vocab_def.txt', 'w')
f.write(string)
f.close()

tfidf_vec = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vec.fit_transform(definitions)
tfidf_matrix_array = tfidf_matrix.toarray()
print(tfidf_matrix_array[0], len(tfidf_matrix_array[0]))

word_vectors = []

for i in range(len(vocab)):
    word = vocab[i]
    vector = tfidf_matrix_array[i]
    str_vector = []
    for j in range(len(vector)):
        str_vector.append(str(vector[j]))
    temp = ' '.join(str_vector)
    word_vector = word + ' ' + temp
    word_vectors.append(word_vector)

string = '\n'.join(word_vectors)

f = open('data/corpus/' + dataset + '_word_vectors.txt', 'w')
f.write(string)
f.close()

word_vector_file = 'data/corpus/' + dataset + '_word_vectors.txt'
_, embd, word_vector_map = loadWord2Vec(word_vector_file)
word_embeddings_dim = len(embd[0])
'''

'''
Word definitions end
'''

In [17]:
# label list
label_set = set()
for doc_meta in tqdm(shuffle_doc_name_list):
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

label_list_str = '\n'.join(label_list)
f = open('data/corpus/' + dataset + '_labels.txt', 'w')
f.write(label_list_str)
f.close()

# x: feature vectors of training docs, no initial features
# slect 90% training set
train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size  # - int(0.5 * train_size)
# different training rates

real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('data/' + dataset + '.real_train.name', 'w')
f.write(real_train_doc_names_str)
f.close()

100%|██████████| 235827/235827 [00:00<00:00, 908205.40it/s]


In [19]:
row_x = []
col_x = []
data_x = []
for i in tqdm(range(real_train_size)):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            # print(doc_vec)
            # print(np.array(word_vector))
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_x.append(doc_vec[j] / doc_len)  # doc_vec[j]/ doc_len

# x = sp.csr_matrix((real_train_size, word_embeddings_dim), dtype=np.float32)
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))

100%|██████████| 169796/169796 [00:44<00:00, 3783.92it/s]


In [20]:
y = []
for i in tqdm(range(real_train_size)):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)
print(y)

100%|██████████| 169796/169796 [00:06<00:00, 27468.47it/s]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [21]:
# tx: feature vectors of test docs, no initial features
test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in tqdm(range(test_size)):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_tx.append(doc_vec[j] / doc_len)  # doc_vec[j] / doc_len

# tx = sp.csr_matrix((test_size, word_embeddings_dim), dtype=np.float32)
tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))

100%|██████████| 47165/47165 [00:12<00:00, 3760.25it/s]


In [22]:
ty = []
for i in tqdm(range(test_size)):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)
print(ty)

100%|██████████| 47165/47165 [00:01<00:00, 40392.67it/s]


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [23]:
# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

In [24]:
for i in tqdm(range(len(vocab))):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

100%|██████████| 1793851/1793851 [00:01<00:00, 1623195.96it/s]


In [25]:
row_allx = []
col_allx = []
data_allx = []

for i in tqdm(range(train_size)):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_allx.append(doc_vec[j] / doc_len)  # doc_vec[j]/doc_len
        
for i in tqdm(range(vocab_size)):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)

allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))

100%|██████████| 1793851/1793851 [05:28<00:00, 5456.29it/s]


In [26]:
ally = []
for i in tqdm(range(train_size)):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

for i in tqdm(range(vocab_size)):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)

100%|██████████| 1793851/1793851 [00:29<00:00, 60448.58it/s]

(169796, 300) (169796, 145) (47165, 300) (47165, 145) (1982513, 300) (1982513, 145)


In [27]:
'''
Doc word heterogeneous graph
'''

# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in tqdm(shuffle_doc_words_list):
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

100%|██████████| 235827/235827 [01:10<00:00, 3353.31it/s] 


In [28]:
word_window_freq = {}
for window in tqdm(windows):
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

100%|██████████| 29259224/29259224 [05:16<00:00, 92442.52it/s]


In [29]:
word_pair_count = {}
for window in tqdm(windows):
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

100%|██████████| 29259224/29259224 [3:33:18<00:00, 2286.19it/s]  


In [30]:
row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in tqdm(word_pair_count):
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

100%|██████████| 285528876/285528876 [12:50<00:00, 370607.69it/s]


In [31]:
# word vector cosine similarity as weights

'''
for i in range(vocab_size):
    for j in range(vocab_size):
        if vocab[i] in word_vector_map and vocab[j] in word_vector_map:
            vector_i = np.array(word_vector_map[vocab[i]])
            vector_j = np.array(word_vector_map[vocab[j]])
            similarity = 1.0 - cosine(vector_i, vector_j)
            if similarity > 0.9:
                print(vocab[i], vocab[j], similarity)
                row.append(train_size + i)
                col.append(train_size + j)
                weight.append(similarity)
'''
# doc word frequency
doc_word_freq = {}

for doc_id in tqdm(range(len(shuffle_doc_words_list))):
    doc_words = shuffle_doc_words_list[doc_id]
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

100%|██████████| 235827/235827 [00:51<00:00, 4548.57it/s]


In [32]:
for i in tqdm(range(len(shuffle_doc_words_list))):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

100%|██████████| 235827/235827 [01:09<00:00, 3369.85it/s]


In [33]:
node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))

In [44]:
# dump objects
f = open("data/ind.{}.x".format(dataset), 'wb')
pkl.dump(x, f)
f.close()

f = open("data/ind.{}.y".format(dataset), 'wb')
pkl.dump(y, f)
f.close()

f = open("data/ind.{}.tx".format(dataset), 'wb')
pkl.dump(tx, f)
f.close()

f = open("data/ind.{}.ty".format(dataset), 'wb')
pkl.dump(ty, f)
f.close()

f = open("data/ind.{}.allx".format(dataset), 'wb')
pkl.dump(allx, f, protocol=4) # Pickle로 4GB 이상의 대용량 파일을 저장하려고 하면 에러가 발생, protocol=4 옵션으로 해결
f.close()

f = open("data/ind.{}.ally".format(dataset), 'wb')
pkl.dump(ally, f)
f.close()

f = open("data/ind.{}.adj".format(dataset), 'wb')
pkl.dump(adj, f)
f.close()